# Shortest Path Algorithms

We survey algorithms solving the shortest path problem:

$$
\begin{alignat*}{3}
& \text{SP} \quad=\quad
    && \text{minimize}   \quad && c^T x \\
&   && \text{subject to} \quad && A x = d \\
&   &&                         && x \geq 0 \\
\end{alignat*}
$$

In [1]:
using DataStructures: BinaryMinHeap
using BenchmarkTools

In [2]:
const Node = Int

struct Arc{T <: Number}
    length::T
    cost::T
end

In [3]:
function _sp_path(parent::Dict{Node, Node}, s::Node, t::Node)
    p = [t]
    while t != s
        push!(p, t)
        t = parent[t]
    end
    reverse!(p)
    p
end

function sp(adj::Dict{Node, Dict{Node, Arc{T}}}, s::Node, t::Node) where {T <: Number}
    frontier = BinaryMinHeap{Tuple{T, Node, Node}}()
    shortest = Dict{Node, T}()
    parent = Dict{Node, Node}()

    push!(frontier, (zero(T), s, s))
    while !isempty(frontier)
        cost, node, pred = pop!(frontier)

        if haskey(shortest, node)
            continue
        end

        parent[node] = pred
        shortest[node] = cost

        if node == t
            return true, cost, _sp_path(parent, s, t)
        else
            for (adjacent, edge) in adj[node]
                push!(frontier, (cost + edge.cost, adjacent, node))
            end
        end
    end

    return false, zero(T), []
end

sp (generic function with 1 method)

In [4]:
function _csp_path(parent::Dict{Tuple{Node, T}, Tuple{Node, T}}, s::Node, t::Node, l::T) where {T <: Number}
    p = [t]
    while t != s
        push!(p, t)
        t, l = parent[t, l]
    end
    reverse!(p)
    p
end

function csp(adj::Dict{Node, Dict{Int, Arc{T}}}, s::Node, t::Node, R::T = Inf) where {T <: Number}
    frontier = BinaryMinHeap{Tuple{T, T, Node, Node}}()
    shortest = Dict{Tuple{Node, T}, T}()
    parent = Dict{Tuple{Node, T}, Tuple{Node, T}}()

    push!(frontier, (zero(T), zero(T), s, -1))
    while !isempty(frontier)
        cost, distance, node, pred = pop!(frontier)

        if haskey(shortest, (node, distance))
            continue
        end

        if pred != -1
            parent[node, distance] = (pred, distance - adj[pred][node].length)
        end
        shortest[node, distance] = cost

        if node == t
            return true, cost, _csp_path(parent, s, t, distance)
        else
            for (adjacent, edge) in adj[node]
                if distance + edge.length <= R
                    push!(frontier, (cost + edge.cost, distance + edge.length, adjacent, node))
                end
            end
        end
    end

    return false, zero(T), []
end

csp (generic function with 2 methods)

In [5]:
n = 90

adj = Dict{Node, Dict{Node, Arc{Int}}}()

function ok(i::Int, j::Int, n::Int)
    0 <= i < n && 0 <= j < n
end

for k in 0:(n * n - 1)
    i, j = divrem(k, n)

    ns = Dict{Node, Arc}()
    for (ni, nj) in [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]
        if ok(ni, nj, n)
            ns[n * ni + nj] = Arc(1, rand(1:5))
        end
    end

    adj[k] = ns
end

In [6]:
s = rand(0:(n * n - 1))
t = rand(0:(n * n - 1))
sp(adj, s, t), csp(adj, s, t, 1000000)

((true, 31, [3765, 3855, 3945, 4035, 4125, 4215, 4305, 4306, 4396, 4486, 4576, 4666, 4756, 4846, 4845, 4845]), (true, 31, [3765, 3855, 3945, 4035, 4125, 4215, 4305, 4306, 4396, 4486, 4576, 4666, 4756, 4846, 4845, 4845]))

In [7]:
@benchmark begin
    s = rand(0:(n * n - 1))
    t = rand(0:(n * n - 1))
    sp(adj, s, t)
end

BenchmarkTools.Trial: 1622 samples with 1 evaluation.
 Range (min … max):  6.750 μs … 12.276 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.081 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.080 ms ±  1.924 ms  ┊ GC (mean ± σ):  0.69% ± 4.87%

  █▅▃▄▃▃▅▃▅▄▂▃▁ ▂▃ ▃▄▃▁▁ ▁▁▄▄▁▄▆▄▃ ▄▁  ▃ ▄▁▃▃▂▁ ▂▂▄ ▁         
  ██████████████████████████████████████▇██████▇█████▆▅▄▃▃▃▃ ▆
  6.75 μs        Histogram: frequency by time        6.88 ms <

 Memory estimate: 2.64 KiB, allocs estimate: 22.

In [8]:
@benchmark begin
    s = rand(0:(n * n - 1))
    t = rand(0:(n * n - 1))
    csp(adj, s, t, 45)
end

BenchmarkTools.Trial: 257 samples with 1 evaluation.
 Range (min … max):  13.875 μs … 34.646 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     20.434 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.501 ms ±  8.502 ms  ┊ GC (mean ± σ):  0.98% ± 5.50%

  ▃                    ▃  ▂ █ ▃▆▃▆ ▂▂▃▆▅ ▃ ▆▃▂▆▂▆ ▅▃█   ▃▅▆ ▃  
  ██▁▇▇▇▁▇▅▄▄▄▄▅▁█▄▁▄▄██▇▅█▅█▇████████████████████████▇▇███▇█ ▅
  13.9 μs         Histogram: frequency by time        32.2 ms <

 Memory estimate: 12.97 KiB, allocs estimate: 29.